In [111]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.cluster import AgglomerativeClustering
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.manifold import TSNE

from time import time
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from sklearn import manifold

In [112]:
game_stats = pd.read_csv('/Users/beh502/ml-practice/clustering/data/Seasons_Stats.csv')
player_stats = pd.read_csv('/Users/beh502/ml-practice/clustering/data/Players.csv')
merged_stats = pd.merge(player_stats, game_stats, on='Player', how='inner')

In [113]:
print "before cleanup, shape of df is : " + str(merged_stats.shape)

#some columns are rubbish, some give us no utility
merged_stats = merged_stats.drop(['collage','birth_city','birth_state','Tm','Unnamed: 0_x','Unnamed: 0_y','blanl','blank2'], axis=1)
merged_stats['player_year'] = merged_stats.Player.map(str) + "-" + merged_stats.Year.map(str)
merged_stats = merged_stats.drop(['Player'], axis=1) #drop now that we ID by player-year

#drop all rows that have some NA values. Only for really old players and they're not elite
merged_stats = merged_stats.dropna(axis=0, how='any')

#dummy encode 'position' column
print "before dummy encoding, num features is: " + str(len(list(merged_stats.columns)))
merged_stats['Pos'] = merged_stats['Pos'].apply(lambda x: 'Wing' if x in ['SG','PG','SF','G','G-F','SG-PG','SG-SF', 'SF-PG','PG-SG','PG-SF','SF-SG'] else 'Big')
merged_stats = pd.get_dummies(merged_stats, prefix=['Pos'], columns=['Pos'])
print "after dummy encoding, num features is: " + str(len(list(merged_stats.columns)))
# added 15 instead of 5 because some players play multiple, ie their value was guard-fwd



before cleanup, shape of df is : (24691, 60)
before dummy encoding, num features is: 52
after dummy encoding, num features is: 53


In [115]:
normalizer = StandardScaler()
clusterer = AgglomerativeClustering(n_clusters=100)

metrics = merged_stats.drop(['player_year'], axis=1).columns.values
merged_stats[metrics] = normalizer.fit_transform(merged_stats[metrics])
merged_stats['cluster'] = clusterer.fit_predict(merged_stats[metrics])

In [116]:
for i in range(0,100):
    print "cluster " + str(i) + " count: " + str(len(merged_stats.loc[merged_stats['cluster'] == i]))

cluster 0 count: 310
cluster 1 count: 127
cluster 2 count: 214
cluster 3 count: 50
cluster 4 count: 82
cluster 5 count: 42
cluster 6 count: 92
cluster 7 count: 342
cluster 8 count: 247
cluster 9 count: 181
cluster 10 count: 202
cluster 11 count: 255
cluster 12 count: 303
cluster 13 count: 178
cluster 14 count: 120
cluster 15 count: 64
cluster 16 count: 148
cluster 17 count: 204
cluster 18 count: 167
cluster 19 count: 377
cluster 20 count: 239
cluster 21 count: 385
cluster 22 count: 147
cluster 23 count: 167
cluster 24 count: 346
cluster 25 count: 250
cluster 26 count: 44
cluster 27 count: 78
cluster 28 count: 357
cluster 29 count: 211
cluster 30 count: 178
cluster 31 count: 275
cluster 32 count: 149
cluster 33 count: 86
cluster 34 count: 18
cluster 35 count: 129
cluster 36 count: 149
cluster 37 count: 106
cluster 38 count: 276
cluster 39 count: 155
cluster 40 count: 163
cluster 41 count: 208
cluster 42 count: 118
cluster 43 count: 159
cluster 44 count: 275
cluster 45 count: 39
cluster 

In [146]:
jordan_93 = merged_stats.loc[merged_stats["player_year"] == "Michael Jordan*-1993.0"]

In [147]:
print jordan_95

        height    weight     born      Year       Age         G        GS  \
9678 -0.097645 -0.599138 -0.69361 -0.803443  0.538754  0.965643  1.730844   

           MP       PER       TS%   ...          AST       STL     BLK  \
9678  1.92949  3.553142  0.810241   ...     2.054015  4.597397  1.0144   

           TOV        PF       PTS             player_year   Pos_Big  \
9678  1.872701  0.895794  4.074049  Michael Jordan*-1993.0 -0.695502   

      Pos_Wing  cluster  
9678  0.695502        6  

[1 rows x 54 columns]


In [148]:
top_cluster = merged_stats.loc[merged_stats["cluster"] == 6]
print top_cluster['player_year']

7482            Magic Johnson*-1987.0
7484            Magic Johnson*-1989.0
7485            Magic Johnson*-1990.0
7486            Magic Johnson*-1991.0
9672           Michael Jordan*-1987.0
9673           Michael Jordan*-1988.0
9674           Michael Jordan*-1989.0
9675           Michael Jordan*-1990.0
9676           Michael Jordan*-1991.0
9677           Michael Jordan*-1992.0
9678           Michael Jordan*-1993.0
9680           Michael Jordan*-1996.0
9681           Michael Jordan*-1997.0
9682           Michael Jordan*-1998.0
14645               Grant Hill-1996.0
14646               Grant Hill-1997.0
14647               Grant Hill-1998.0
14649               Grant Hill-2000.0
15657              Kobe Bryant-2001.0
15658              Kobe Bryant-2002.0
15659              Kobe Bryant-2003.0
15660              Kobe Bryant-2004.0
15662              Kobe Bryant-2006.0
15663              Kobe Bryant-2007.0
15664              Kobe Bryant-2008.0
15665              Kobe Bryant-2009.0
15666       

In [ ]:
# DONE WITH AGGLOMERATIVE CLUSTERING

In [80]:
x_tsne = TSNE(learning_rate=100).fit_transform(merged_stats[metrics])

In [84]:
def plot_clustering(X_red, X, labels, title=None):
    x_min, x_max = np.min(X_red, axis=0), np.max(X_red, axis=0)
    X_red = (X_red - x_min) / (x_max - x_min)

    plt.figure(figsize=(6, 4))
    for i in range(X_red.shape[0]):
        plt.text(X_red[i, 0], X_red[i, 1], str(y[i]),
                 color=plt.cm.spectral(labels[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title, size=17)
    plt.axis('off')
    plt.tight_layout()


In [85]:
# 2D embedding of the digits dataset
print("Computing embedding")
X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(merged_stats[metrics])
print("Done.")

from sklearn.cluster import AgglomerativeClustering

for linkage in ('ward', 'average', 'complete'):
    clustering = AgglomerativeClustering(linkage=linkage, n_clusters=10)
    t0 = time()
    clustering.fit(X_red)
    print("%s : %.2fs" % (linkage, time() - t0))

    plot_clustering(X_red, X, clustering.labels_, "%s linkage" % linkage)


plt.show()

Computing embedding
Done.
ward : 6.14s


NameError: name 'X' is not defined

In [ ]:
# hierarchial (agglomerative)
# CURE (compare to hierarchial)
# spectral
# KNN
# SOM (self organizing maps)
